# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../01-TheField/02-Python/02-PythonAdvanced/05-Scraping/03-apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [32]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{status_url}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [33]:
# Set the countries_url variable (1 line)
countries_url = "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{countries_url}")
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(countries, req.status_code)

{'message': 'The cookie is missing'} 403


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [40]:
# Set the cookie_url variable (1 line)
cookie_url = "/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(f"{root_url}{cookie_url}").cookies
cookies

<RequestsCookieJar[Cookie(version=0, name='user_cookie', value='370f1228-b6a0-4d9d-88ef-0cef7a677f3e', port=None, port_specified=False, domain='country-leaders.onrender.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'SameSite': 'lax'}, rfc2109=False)]>

Try to query the countries endpoint using the cookie, save the output and print it.

In [41]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
# display the countries variable (1 line)
countries

['ma', 'be', 'us', 'fr', 'ru']

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [42]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}{leaders_url}")
# display the leaders variable (1 line)
leaders

<Response [403]>

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [43]:
from pprint import pprint

# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}{leaders_url}", params={"country": "fr"}, cookies=cookies).json()
# display the leaders variable (1 line)
leaders

[{'id': 'Q157',
  'first_name': 'François',
  'last_name': 'Hollande',
  'birth_date': '1954-08-12',
  'death_date': None,
  'place_of_birth': 'Rouen',
  'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande',
  'start_mandate': '2012-05-15',
  'end_mandate': '2017-05-14'},
 {'id': 'Q329',
  'first_name': 'Nicolas',
  'last_name': 'Sarkozy',
  'birth_date': '1955-01-28',
  'death_date': None,
  'place_of_birth': 'Paris',
  'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy',
  'start_mandate': '2007-05-16',
  'end_mandate': '2012-05-15'},
 {'id': 'Q2038',
  'first_name': 'François',
  'last_name': 'Mitterrand',
  'birth_date': '1916-10-26',
  'death_date': '1996-01-08',
  'place_of_birth': 'Jarnac',
  'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand',
  'start_mandate': '1981-05-21',
  'end_mandate': '1995-05-17'},
 {'id': 'Q2042',
  'first_name': 'Charles',
  'last_name': 'de Gaulle',
  'birth_date': '1890-11-22',
  'death_date': '

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [44]:
# 4 lines
leaders_per_country = {}
for country in countries:
    country_leaders = requests.get(f"{root_url}{leaders_url}", params={"country": {country}}, cookies=cookies).json()
    leaders_per_country[country] = country_leaders
pprint(leaders_per_country)

{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
         'place_of_birth': 'Dendermonde',
         'start_mandate': '1999-07-12',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'},
        {'birth_date': '1960-10-06',
         'death_date': None,
         'end_mandate': '2011-12-06',
         'first_name': 'Yves',
         'id': 'Q12981',
         'last_name': 'Leterme',
         'place_of_birth': 'Wervik',
         'start_mandate': '2009-11-25',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme'},
        {'birth_date': '1947-10-31',
         'death_date': None,
         'end_mandate': '2009-11-25',
         'first_name': 'Herman',
         'id': 'Q12983',
         'last_name': 'None',
         'place_of_birth': 'Etterbeek',
         'start_mandate': '2008-12-30',
         'wikipedia_url': 'http

In [45]:
# or 1 line
from pprint import pprint
leader_per_country = {country: requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": {country}}).json()for country in countries}
pprint(leader_per_country)

{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
         'place_of_birth': 'Dendermonde',
         'start_mandate': '1999-07-12',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'},
        {'birth_date': '1960-10-06',
         'death_date': None,
         'end_mandate': '2011-12-06',
         'first_name': 'Yves',
         'id': 'Q12981',
         'last_name': 'Leterme',
         'place_of_birth': 'Wervik',
         'start_mandate': '2009-11-25',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme'},
        {'birth_date': '1947-10-31',
         'death_date': None,
         'end_mandate': '2009-11-25',
         'first_name': 'Herman',
         'id': 'Q12983',
         'last_name': 'None',
         'place_of_birth': 'Etterbeek',
         'start_mandate': '2008-12-30',
         'wikipedia_url': 'http

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [46]:
# < 15 lines
def get_leaders():
    import requests

    # define urls
    root_url = "https://country-leaders.onrender.com"
    cookie_url, countries_url, leaders_url = "/cookie", "/countries", "/leaders"

    # get the cookies
    cookies = requests.get(f"{root_url}{cookie_url}").cookies

    # loop over the countries and add leaders to dictionnary leader_per_country
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
    leader_per_country = {country: requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country}).json() for country in countries}

    return leader_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [47]:
# 2 lines
import requests
from pprint import pprint 
leader_per_country = get_leaders()
pprint(leader_per_country)

{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
         'place_of_birth': 'Dendermonde',
         'start_mandate': '1999-07-12',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'},
        {'birth_date': '1960-10-06',
         'death_date': None,
         'end_mandate': '2011-12-06',
         'first_name': 'Yves',
         'id': 'Q12981',
         'last_name': 'Leterme',
         'place_of_birth': 'Wervik',
         'start_mandate': '2009-11-25',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme'},
        {'birth_date': '1947-10-31',
         'death_date': None,
         'end_mandate': '2009-11-25',
         'first_name': 'Herman',
         'id': 'Q12983',
         'last_name': 'None',
         'place_of_birth': 'Etterbeek',
         'start_mandate': '2008-12-30',
         'wikipedia_url': 'http

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [48]:
# 3 lines
country_name, last_name = 'be', 'Wilmès'

url_wiki = next(person['wikipedia_url'] for person in leader_per_country[country_name] if person['last_name'] == last_name)

wiki_data = requests.get(url_wiki).text

print(wiki_data)

# for person in leader_per_country['fr']:
#     if person['last_name'] == 'de Gaulle':
#         person_url = person['wikipedia_url']



<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Sophie Wilmès - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-li

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [49]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_data, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Sophie Wilmès - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disable

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [50]:
# 2 lines
from bs4 import BeautifulSoup
paragraphs = soup.find("table", attrs={"class" : "infobox"}).find_next_siblings("p")
for element in paragraphs:
    print(element.text)


Sophie Wilmès (Elsene, 15 januari 1975) is een Belgisch politica van de MR. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke premier van België.[1] Vanaf 1 oktober 2020 was ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer.



If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [51]:
# <10 lines
first_paragraph = ""
for paragraph in paragraphs:
    first_paragraph += paragraph.text
    break

print(first_paragraph)


Sophie Wilmès (Elsene, 15 januari 1975) is een Belgisch politica van de MR. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke premier van België.[1] Vanaf 1 oktober 2020 was ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [52]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    wiki_data = requests.get(wikipedia_url).text
    soup = BeautifulSoup(wiki_data, 'html.parser')
    paragraphs = soup.find("table", attrs={"class" : "infobox"}).find_next_siblings("p")
    first_paragraph = ""
    for paragraph in paragraphs:
        first_paragraph += paragraph.text
        break
    return first_paragraph

from bs4 import BeautifulSoup
country_name, last_name = 'fr', 'Sarkozy'
wikipedia_url = next(person['wikipedia_url'] for person in leader_per_country[country_name] if person['last_name'] == last_name)
print(get_first_paragraph(wikipedia_url))


https://fr.wikipedia.org/wiki/Nicolas_Sarkozy
Nicolas Sarközy de Nagy-Bocsa[d], dit Nicolas Sarkozy (/ni.kɔ.la saʁ.kɔ.zi/[e] ; Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012.



In [53]:
# Test: 3 lines
def get_first_paragraphv2(wikipedia_url):
    print(wikipedia_url)
    first_paragraph = next(paragraph.text for paragraph in BeautifulSoup(requests.get(wikipedia_url).text, 'html.parser').find("table", attrs={"class" : "infobox"}).find_next_siblings("p"))
    return first_paragraph

from bs4 import BeautifulSoup
country_name, last_name = 'fr', 'Hollande'
wikipedia_url = next(person['wikipedia_url'] for person in leader_per_country[country_name] if person['last_name'] == last_name)
sentence = get_first_paragraphv2(wikipedia_url)
print(sentence)

https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Hollande [fʁɑ̃swa ɔlɑ̃d][n 2] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [54]:
# 3 lines
import re

sentence = "Sophie Wilmès (Elsene, 15 januari 1975) is een Belgisch politica van de MR. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke premier van België.[1] Vanaf 1 oktober 2020 was ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer."
paragraph_clean = re.sub(r'\[.*?\]|\([^)]*Écouter\)', '', sentence)


print(paragraph_clean)

Sophie Wilmès (Elsene, 15 januari 1975) is een Belgisch politica van de MR. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke premier van België. Vanaf 1 oktober 2020 was ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [55]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    wiki_data = requests.get(wikipedia_url).text
    soup = BeautifulSoup(wiki_data, 'html.parser')
    first_paragraph = next(paragraph.text for paragraph in soup.find("table", attrs={"class" : "infobox"}).find_next_siblings("p"))
    paragraph_clean = re.sub(r' \[.*?\]\[.*?\]|\([^)]*Écouter\)|\ Écouter|\[\w]', '', first_paragraph)
    return paragraph_clean

import re
from bs4 import BeautifulSoup

# use bith_date for 'ma'
country_name, last_name, birth_date = 'fr', 'Hollande', '1963-08-21'
wikipedia_url = next(person['wikipedia_url'] for person in leader_per_country[country_name] if person['last_name'] == last_name)
print(get_first_paragraph(wikipedia_url))



https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Hollande, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [56]:
# < 20 lines
def get_leaders():
    import requests
    # define urls
    root_url = "https://country-leaders.onrender.com"
    cookie_url, countries_url, leaders_url = "/cookie", "/countries", "/leaders"
    # get the cookies
    cookies = requests.get(f"{root_url}{cookie_url}").cookies
    # loop over the countries and add leaders to dictionnary leader_per_country
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
    leader_per_country = {country: requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country}).json() for country in countries}
    return leader_per_country

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    wiki_data = requests.get(wikipedia_url).text
    soup = BeautifulSoup(wiki_data, 'html.parser')
    first_paragraph = next(paragraph.text for paragraph in soup.find("table", attrs={"class" : "infobox"}).find_next_siblings("p"))
    paragraph_clean = re.sub(r' \[.*?\]\[.*?\]|\([^)]*Écouter\)|\ Écouter|\[\w]', '', first_paragraph)
    paragraph_clean = re.sub(r'\s+,', ',', paragraph_clean)
    return paragraph_clean



## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [66]:

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookie_url, countries_url, leaders_url = "/cookie", "/countries", "/leaders"
    # Get the cookies
    cookies = requests.get(f"{root_url}{cookie_url}").cookies
    # Get the list of countries
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
    # Dictionary leaders per country
    leader_per_country = {}
    for country in countries:
        leaders = requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country}).json()
        # Add a check to ensure leaders is a list
        if isinstance(leaders, list):
            for leader in leaders:
                wikipedia_url = leader.get('wikipedia_url')
                if wikipedia_url:
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    leader['first_paragraph'] = first_paragraph
        else:
            print(f"Unexpected response format for country {country}: {leaders}")
        leader_per_country[country] = leaders
    return leader_per_country

In [67]:
# Check the output of your function (2 lines)
# Call the function and print the result
leader_per_country = get_leaders()
pprint(leader_per_country)

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD,_%D

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [68]:
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    countries_url, leaders_url = "/countries", "/leaders"
    cookies = requests.get(f"{root_url}{cookie_url}").cookies
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
    leader_per_country = {}

    for country in countries:
        try:
            response = requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country})
            if response.status_code == 403:  # Forbidden, likely due to cookie issues
                cookies = requests.get(f"{root_url}{cookie_url}").cookies
                response = requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country})
            leaders = response.json()
            for leader in leaders:
                wikipedia_url = leader.get('wikipedia_url')
                if wikipedia_url:
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    leader['first_paragraph'] = first_paragraph
            leader_per_country[country] = leaders
        except requests.exceptions.RequestException as e:
            print(f"Request failed for country {country}: {e}")
            leader_per_country[country] = []
    return leader_per_country

Check the output of your function again.

In [69]:
# Check the output of your function (1 line)
leader_per_country = get_leaders()
from pprint import pprint
pprint(leader_per_country)

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD,_%D

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [70]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session):
    print(wikipedia_url)  # keep this for the rest of the notebook
    wiki_data = session.get(wikipedia_url).text
    soup = BeautifulSoup(wiki_data, 'html.parser')
    first_paragraph = next(paragraph.text for paragraph in soup.find("table", attrs={"class": "infobox"}).find_next_siblings("p"))
    paragraph_clean = re.sub(r' \[.*?\]\[.*?\]|\([^)]*Écouter\)|\ Écouter|\[\w]', '', first_paragraph)
    paragraph_clean = re.sub(r'\s+,', ',', paragraph_clean)
    return paragraph_clean


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [71]:
# <25 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookie_url = "/cookie"
    countries_url, leaders_url = "/countries", "/leaders"
    cookies = requests.get(f"{root_url}{cookie_url}").cookies
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()
    leader_per_country = {}

    with requests.Session() as session:
        for country in countries:
            try:
                response = requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country})
                if response.status_code == 403:  # Forbidden, likely due to cookie issues
                    cookies = requests.get(f"{root_url}{cookie_url}").cookies
                    response = requests.get(f"{root_url}{leaders_url}", cookies=cookies, params={"country": country})
                leaders = response.json()
                for leader in leaders:
                    wikipedia_url = leader.get('wikipedia_url')
                    if wikipedia_url:
                        first_paragraph = get_first_paragraph(wikipedia_url, session)
                        leader['first_paragraph'] = first_paragraph
                leader_per_country[country] = leaders
            except requests.exceptions.RequestException as e:
                print(f"Request failed for country {country}: {e}")
                leader_per_country[country] = []
    return leader_per_country



Test your new functions.



In [72]:
# Call the function and print the result
leader_per_country = get_leaders()
pprint(leader_per_country)

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87
https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD,_%D

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [73]:
# 3 lines
import json
with open('leaders.json', 'w') as f:
    json.dump(leader_per_country, f, indent=4)


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [75]:
# 3 lines
saved_data = json.load(open("leaders.json", 'r'))
if saved_data == leader_per_country:
    print("Variables are identical. Data integrity verified.")

Variables are identical. Data integrity verified.


Make a function `save(leaders_per_country)` to call this code easily.

In [82]:
# 3 lines
def save(leader_per_country):
    with open('leaders.json', 'w') as f:
        json.dump(leader_per_country, f, indent=4)



In [83]:
# Call the function (1 line)
save(leader_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!